In [3]:
import pandas as pd
from io import StringIO

df = pd.read_csv('sentiment.csv')

col = ['data', 'title']
df = df[col]
df = df[pd.notnull(df['data'])]
df.columns = ['data', 'title']

print(df)


                                                    data  \
0      রুদ্র জাহেদ ভাই খুব ভাল লাগল আপনার মন্তব্যে অন...   
1      যারা ব্লগ দিয়ে ইন্টারনেট চালানোর দাবি করেছিল ২...   
2                                      থ্যাংকস সুমন ভাই।   
3                                      থ্যাংকস সুমন ভাই।   
4                                  সময়ের নিষ্ঠ প্রতিবাদ।   
5      সমস্যা বটে । আর একটা কারন হতে পারে "নতুন ব্লগ ...   
6      ভাল লিখেছেন। ধর্ষন খুব খারাপ কাজ।আমাদের সমাজে ...   
7      আমিও চাই সব মানুষ মানুষ হিসেবে বাঁচুক । কিন্তু...   
8      ভাল লিখেছেন। ধর্ষন খুব খারাপ কাজ।আমাদের সমাজে ...   
9                                      থ্যাংকস সুমন ভাই।   
10     সমস্যা বটে । আর একটা কারন হতে পারে "নতুন ব্লগ ...   
11     রুদ্র জাহেদ ভাই খুব ভাল লাগল আপনার মন্তব্যে অন...   
12     আমিও চাই সব মানুষ মানুষ হিসেবে বাঁচুক । কিন্তু...   
13     আমিও চাই সব মানুষ মানুষ হিসেবে বাঁচুক । কিন্তু...   
14                                     থ্যাংকস সুমন ভাই।   
15     আমিও চাই সব মানুষ মানুষ হিসেবে বা

In [6]:

df['category_id'] = df['title'].factorize()[0]
category_id_df = df[['title', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'title']].values)
# print(df.head())




In [7]:

##data absorbing done

##feature extraction begins here


##tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.data).toarray()
labels = df.category_id
print(features.shape)


(27731, 20759)


In [8]:

##uni and bi-grams

from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  # print("# '{}':".format(Product))
  # print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  # print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))



In [9]:

##first test model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['data'], df['title'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [10]:

##first test models ends here

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])



In [11]:
print(cv_df.groupby('model_name').accuracy.mean())


model_name
LinearSVC                 0.239011
LogisticRegression        0.393388
MultinomialNB             0.393317
RandomForestClassifier    0.400058
Name: accuracy, dtype: float64
